# Graph scoping - členění Grafu za pomoci name_scope

tf.name_scope přidává všem objektů jmenný prefix stejný jako parametr name_scope. Používání name_scope není povinné, ale pomůže Vám zpřehlednit / oddělit / jednotlivé funkční celky grafu, což oceníte v případě, že se na graf výpočtu budete dívat v tensorboardu (viz. Co je  Tensorboard). Pomocí jménného scope se také můžete odkazovat na konkrétní Tensory použité ve scope.  

In [2]:
import tensorflow as tf

In [3]:
with tf.variable_scope("one"):
    a = tf.get_variable("v", [1])
    print(a.name)

one/v:0


Pokud bychom použili ve stejném variable_scope stejné jméno Tensoru Variable, pak je vrácena exception - ValueError: Variable one/v already exists. Parametr reuse = True však umožňuje název znovu použít. V tom případě budou objekty totožné.

In [13]:
with tf.variable_scope("one", reuse = True):
    c = tf.get_variable("v", [1])
    print(a.name)

one/v:0


In [14]:
a is c

True

Získání hodnoty Variable <b>a</b> pro definici <b>b</b> prostřednictvím name-scope.

In [ ]:
with tf.name_scope('operations'):
    ac = tf.multiply(a, c, name='multiply_ac')
    bc = tf.multiply(b, c, name='multiply_bc')
    de = tf.add(ac, bc, name='sumof_ac_bc')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("./tensorboard_example", sess.graph)
    data_feed = [[1., 2., 3., 4.]]
    ac_res, bc_res, de_res = sess.run([ac, bc, de], feed_dict={c: data_feed})
    print("a.eval(): ", a.eval(), " a.name: ", a.name)
    print("b.eval(): ", b.eval(), " a.name: ", b.name)
    print("ac_res: ", ac_res)
    print("bc_res: ", bc_res)
    print("de_res: ", de_res)
    print("shape of a:", a.get_shape())
    print("shape of b:", b.get_shape())
    print("shape of de:", de.get_shape())

# Co je to ten Tensorboard?

Podívejme se teď na Tensorboard. Tensorboard slouží především k vizualizaci Grafu a vývoje hodnot prvků v grafu. Spouští na localhostu vlastní webovou službu.
tensorboard.exe --logdir=".\tensorboard_example" --port 9000